# Init

In [1]:
!sudo apt update
!pip install pyspark

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Ign:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,105 kB]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
G

In [2]:
!ls -la

total 72
drwxr-xr-x 1 root root  4096 Nov  3 11:23 .
drwxr-xr-x 1 root root  4096 Nov  3 11:17 ..
-rw-r--r-- 1 root root 19173 Nov  3 11:23 attendance.parquet
drwxr-xr-x 4 root root  4096 Oct 31 13:28 .config
-rw-r--r-- 1 root root 32976 Nov  3 11:23 employee.parquet
drwxr-xr-x 1 root root  4096 Oct 31 13:28 sample_data


In [3]:
# Добавляем Hive таблицу employee
from pyspark.sql import SparkSession
spark = SparkSession \
       .builder \
       .master('local[*]') \
       .appName("Our First Spark Example") \
       .enableHiveSupport() \
       .getOrCreate()
spark.read.parquet('employee.parquet').repartition(1).write.saveAsTable("employee")
spark.read.parquet('attendance.parquet').repartition(1).write.saveAsTable("attendance")
spark.stop()

# Старт сессии и импорт

Импортируем необходимые библиотеки

Документация PySpark: https://spark.apache.org/docs/latest/api/python/index.html

Документация PySpark SQL Functions: https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/functions.html

In [4]:
from pyspark.sql import DataFrame, SparkSession
import pyspark.sql.functions as F

In [5]:
# Создание спарк сессии
spark = SparkSession \
       .builder \
       .master('local[*]') \
       .appName("Spark Practice") \
       .enableHiveSupport() \
       .getOrCreate()

spark

#С какими файлами работаем?

1) Будем работать с Hive-таблицами

2) Данные о сотрудниках в компании

In [6]:
# Смотрим, какие таблицы есть в Hive etastore
spark.sql("show tables").show()

+---------+----------+-----------+
|namespace| tableName|isTemporary|
+---------+----------+-----------+
|  default|attendance|      false|
|  default|  employee|      false|
+---------+----------+-----------+



In [ ]:
# Есть 2 способа работать с данными в Spark
# 1) Через Spark SQL: spark.sql(" ... ").show()
# 2) Через Spark DataFrame API: spark.read.table(" ... ").show()

In [10]:
# Читаем таблицу employee и выводим ее схему
spark.read.table("employee").printSchema()

root
 |-- Id: integer (nullable = true)
 |-- LastName: string (nullable = true)
 |-- FirstName: string (nullable = true)
 |-- MiddleName: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Department: string (nullable = true)
 |-- Position: string (nullable = true)
 |-- Grade: string (nullable = true)
 |-- HireDate: date (nullable = true)
 |-- OfferAmount: double (nullable = true)
 |-- Education: string (nullable = true)
 |-- CurrentStatus: string (nullable = true)



In [11]:
# Читаем таблицу attendance и выводим ее 5 первых строк
spark.read.table("employee").show(5)

+---+--------+---------+------------+------+---+----------+--------------------+------+----------+-----------+-------------------+-------------+
| Id|LastName|FirstName|  MiddleName|Gender|Age|Department|            Position| Grade|  HireDate|OfferAmount|          Education|CurrentStatus|
+---+--------+---------+------------+------+---+----------+--------------------+------+----------+-----------+-------------------+-------------+
|  1| Абрамов|   Самсон|Валерьянович|Female| 36| Analytics|     Аналитик данных|Стажер|2024-06-14|   89178.63|       Магистратура|       Уволен|
|  2| Абрамов|   Анжела|   Тихонович|Female| 54|        IT|Продуктовый аналитик|Стажер|2015-12-20|  101972.96|       Магистратура|      Активен|
|  3| Абрамов|Харлампий|   Елисеевич|Female| 38|        IT|  Разработчик систем|Стажер|2019-09-30|   88074.95|Среднее специальное|       Уволен|
|  4| Абрамов|   Ерофей|Валентинович|  Male| 34| Analytics|Разработчик баз д...|Junior|2024-03-08|  120694.35|Среднее специальное|

In [7]:
# Читаем таблицу attendance и выводим ее схему
spark.read.table("attendance").printSchema()

root
 |-- Id: integer (nullable = true)
 |-- Date: date (nullable = true)
 |-- WorkedHours: double (nullable = true)
 |-- AbsenceReason: string (nullable = true)



In [9]:
# Читаем таблицу attendance и выводим ее 5 первых строк
spark.read.table("attendance").show(5)

+---+----------+-----------+--------------+
| Id|      Date|WorkedHours| AbsenceReason|
+---+----------+-----------+--------------+
|229|2024-05-30|        3.5|          NULL|
|642|2024-09-12|       11.6|          NULL|
|300|2024-04-30|        6.9|          NULL|
|614|2024-03-04|        0.0|Personal Leave|
|399|2024-02-06|        9.9|          NULL|
+---+----------+-----------+--------------+
only showing top 5 rows



In [ ]:
# Вывести среднюю, минимальную и максимальную зп у всех сотрудников (округлить до 2 знаков после запятой)
spark.sql("""

          """).show()

In [ ]:
# Вывести среднюю зп у Активных сотрудников в зависимости от Grade (округлить до 2 знаков после запятой)
spark.sql("""

          """).show()

In [ ]:
# Посчитать средний возраст всех сотрудников.
spark.sql("""

          """).show()

In [ ]:
# Посчитать количество мужчин и женщин в компании.
spark.sql("""

          """).show()

In [ ]:
# Посчитать количество сотрудников по отделам (Department).
spark.sql("""

          """).show()

In [ ]:
# Определить, как долго сотрудники работают в компании, используя разницу между текущей датой и датой их найма.
# from pyspark.sql.functions import current_date, datediff
spark.sql("""

          """).show()

In [ ]:
# Определить средний доход специалистов в зависимости от уровня образования
spark.sql("""

          """).show()

In [ ]:
# Посчитать среднюю продолжительность работы сотрудников в компании (с учетом даты найма).
spark.sql("""

          """).show()

In [ ]:
# Преобразовать CurrentStatus в Int (Уволен = 0, Активен = 1)
spark.sql("""

          """).show()

In [ ]:
# Посчитать среднее количество отработанных часов для всех сотрудников.
spark.sql("""

          """).show()

In [ ]:
# Посчитать среднее количество отработанных часов в зависимости от Грейда.
spark.sql("""

          """).show()

In [ ]:
# Посчитать среднее количество отработанных часов в зависимости от Возраста.
spark.sql("""

          """).show()

In [ ]:
# Посчитать среднее количество отработанных часов в зависимости от Пола сотрудника.
spark.sql("""

          """).show()